In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..'))

# NOTE: At some stage we might want to convert this to a HF dataset and use the HF dataset API
from dataset.constants import CHALLENGE_DATASET
dataset_save_dir = os.path.join(os.path.dirname(''), "..", 'dataset', "data")

print(CHALLENGE_DATASET.keys())
demo_dataset = CHALLENGE_DATASET['FacebookAI-xlm-roberta-base']
save_folder = demo_dataset.rsplit('/', 1)[-1][:-4]


dict_keys(['demo', 'small', 'large', 'test', 'word2vec', 'image_embeddings', 'contrastive_vector', 'google-bert-base-multilingual-cased', 'FacebookAI-xlm-roberta-base'])


In [18]:
from tqdm import tqdm
import requests
import zipfile

def download_file(url, dest):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024*1024  # 1 Kibibyte

    file = open(dest, 'wb')
    for data in tqdm(
            response.iter_content(block_size),
            total=total_size // block_size,
            unit='MiB',
            unit_scale=True,
            unit_divisor=1024,
            
    ):
        file.write(data)
    file.close()

def unzip_file(zip_path, dest_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)


# Download the dataset
os.makedirs(dataset_save_dir, exist_ok=True)
download_file(demo_dataset, os.path.join(dataset_save_dir, save_folder + ".zip"))
unzip_file(os.path.join(dataset_save_dir, save_folder + ".zip"), dataset_save_dir)


342MiB [00:28, 12.2MiB/s]                         


In [1]:
import polars as pl
dataset = pl.scan_parquet('../dataset/data/FacebookAI_xlm_roberta_base/xlm_roberta_base.parquet').collect()

In [2]:
selector = dataset["article_id"].is_in([9768820]).to_numpy()
dataset

article_id,FacebookAI/xlm-roberta-base
i32,list[f32]
3000022,"[0.102449, 0.101148, … -0.020715]"
3000063,"[0.107297, 0.103073, … 0.004873]"
3000613,"[0.125139, 0.124621, … -0.05177]"
3000700,"[0.105697, 0.076335, … -0.034872]"
3000840,"[0.098175, 0.114629, … -0.024436]"
3001278,"[0.070491, 0.075925, … -0.015167]"
3001299,"[0.104734, 0.104024, … -0.020324]"
3001353,"[0.082843, 0.117304, … -0.010013]"
3001457,"[0.11277, 0.109303, … 0.000977]"


In [3]:
dataset["FacebookAI/xlm-roberta-base"].to_numpy()[selector]

array([array([ 1.41411245e-01,  1.60694391e-01,  6.88351393e-02,  1.46988407e-02,
               1.13561422e-01, -3.84560265e-02,  2.54990775e-02, -5.10339998e-02,
               1.19911015e-01, -1.67081878e-01,  4.05034758e-02,  1.38458431e-01,
              -1.12315960e-01, -1.57548562e-02,  1.19955875e-02,  4.16031592e-02,
              -6.56966493e-02, -1.70452148e-02,  1.12449743e-01,  7.29211867e-02,
               4.82784286e-02,  2.08646385e-03,  1.22568108e-01,  1.05366521e-01,
              -8.40007588e-02,  4.78900112e-02, -3.63052264e-02, -2.93072388e-02,
               8.81975442e-02, -1.11660745e-03,  1.04680806e-01, -1.60349868e-02,
               3.35927308e-02,  7.63882771e-02,  1.42348737e-01, -1.19436195e-03,
              -2.01697901e-01, -9.45579037e-02,  8.08745623e-02,  4.86845486e-02,
               3.54028828e-02,  2.35228434e-01, -2.82336492e-02, -4.59266230e-02,
               4.47614565e-02, -1.01837069e-01, -1.04254708e-01, -8.83764401e-02,
              -1